In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [10]:
run_path = '/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_support_code/TRENDY_run1/'


output_paths = glob.glob(run_path+'var_out/*')
input_paths = glob.glob(run_path+'clm_out/*')

print(len(output_paths))
print(len(input_paths))

5000
3276


In [28]:

output_group = [re.findall('[0-9]+$', w) for w in output_paths]
output_group = [item for sublist in output_group for item in sublist]

output_var = [re.findall('GPP|NPP|VEGC|SOILORGC|VEGINGPP|FRDL|FCO2|TEMP|FH2O|FO3', w) for w in output_paths]
output_var = [item for sublist in output_var for item in sublist]

input_group = [re.findall('[0-9]+$', w) for w in input_paths]
input_group = [item.upper() for sublist in input_group for item in sublist]

input_var = [re.findall('PREC|NIRR|PAR|TAIR|VPR|WS10', w) for w in input_paths]
input_var = [item for sublist in input_var for item in sublist]


In [29]:
print(len(input_paths))
print(len(input_group))
print(len(input_var))

3276
3276
3276


In [30]:

input_data = pd.DataFrame(
{'input_paths': input_paths,
'input_group': input_group,
 'input_var': input_var
}
)

output_data = pd.DataFrame(
{'output_paths': output_paths,
'output_group': output_group,
 'output_var': output_var
}
)


In [31]:
input_data

,input_paths,input_group,input_var
0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,55818,PAR
1,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,43470,TAIR
2,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,54684,NIRR
3,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,32130,PAR
4,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,36414,PREC
...,...,...,...
3271,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,40194,PAR
3272,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,23688,TAIR
3273,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,34650,PREC
3274,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,47502,PREC


In [33]:

input_data = pd.pivot(input_data, index = 'input_group', columns = 'input_var', values = 'input_paths').reset_index()

read_in_data = input_data.merge(output_data, left_on = 'input_group', right_on = 'output_group')




In [34]:
read_in_data

,input_group,NIRR,PAR,PREC,TAIR,VPR,WS10,output_paths,output_group,output_var
0,0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,0,NPP
1,0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,0,FCO2
2,0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,0,SOILORGC
3,0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,0,TEMP
4,0,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,0,FRDL
...,...,...,...,...,...,...,...,...,...,...
4995,9954,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,9954,FH2O
4996,9954,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,9954,SOILORGC
4997,9954,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,9954,NPP
4998,9954,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,/home/smmrrr/TEM/TEM_Runs/TEM_parallel_run_sup...,9954,VEGINGPP


In [35]:

read_in_data.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/Aggregate_TEM/all_parallel_variable_info.csv', index=False)
# tt = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/test_parallel_run/all_parallel_variable_info.csv')


In [13]:
tt.loc[int('0')]

input_group                   0
CLDS                  CLDS.HVD0
PAR                    PAR.HVD0
PREC                  PREC.HVD0
TAIR                  TAIR.HVD0
VPR                    VPR.HVD0
output_paths    SOLN_LULCC.csv0
output_group                  0
output_var                 SOLN
Name: 0, dtype: object